## Import Library

In [1]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Import Dataset

In [2]:
#Generate Data weather four locations
def generate_weather_data(start_date, num_days, num_variables):
    # Generate dates
    dates = [start_date + timedelta(days=i) for i in range(num_days)]
    
    # Generate random data for weather variables
    data = {f"Variable_{i+1}": np.round(np.random.uniform(0, 100, num_days), 2) for i in range(num_variables)}
    
    # Create DataFrame
    df = pd.DataFrame(data)
    df.insert(0, "Date", dates)
    return df

# Parameters
start_date = datetime(2016, 1, 1)
num_days = 1460
num_variables = 10
num_files = 4
input_steps = 45
output_steps = 14
output_dir = "synthetic_data"

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)

# Generate and save CSV files
for i in range(1, num_files + 1):
    weather_data = generate_weather_data(start_date, num_days, num_variables)
    file_name = os.path.join(output_dir, f"weather_data_{i}.csv")
    weather_data.to_csv(file_name, index=False)
    print(f"File saved: {file_name}")

File saved: synthetic_data/weather_data_1.csv
File saved: synthetic_data/weather_data_2.csv
File saved: synthetic_data/weather_data_3.csv
File saved: synthetic_data/weather_data_4.csv


In [3]:
# #Generate data for engine data
# num_samples = 1000
# output_dir = "synthetic_data"  # Reusing the same directory
# file_name = "engine_data.csv"

# # Ensure output directory exists
# os.makedirs(output_dir, exist_ok=True)

# # Generate random engine data
# resistance = np.random.rand(num_samples, 1).astype(np.float32)
# power = np.random.rand(num_samples, 1).astype(np.float32)
# torque = np.random.rand(num_samples, 1).astype(np.float32)
# total_time = np.random.rand(num_samples, 1).astype(np.float32)
# distance = np.random.rand(num_samples, 1).astype(np.float32)
# total_energy = np.random.rand(num_samples, 1).astype(np.float32)

# # Combine into a single DataFrame
# engine_data = pd.DataFrame({
#     "resistance": resistance.flatten(),
#     "power": power.flatten(),
#     "torque": torque.flatten(),
#     "total_time": total_time.flatten(),
#     "distance": distance.flatten(),
#     "total_energy": total_energy.flatten()
# })

# # Save as a single CSV file
# file_path = os.path.join(output_dir, file_name)
# engine_data.to_csv(file_path, index=False)
# print(f"File saved: {file_path}")

## Train Test Split

### Function

def load_weather_data_as_timeseries(data_dir, num_variables=10):
    time_series_list = []
    for i in range(1, 5):  # Assuming 4 weather CSV files
        file_path = os.path.join(data_dir, f"weather_data_{i}.csv")
        df = pd.read_csv(file_path)
        time_series_list.append(df.iloc[:, 1:].to_numpy())  # Drop Date column, keep variables
    return time_series_list

# Load engine data
def load_engine_data(data_dir, file):
    engine_file_path = os.path.join(data_dir, file)
    engine_data = pd.read_csv(engine_file_path)
    return engine_data

# Min-Max scaling for each time series
def scale_time_series(time_series_list):
    scalers = []
    scaled_series_list = []
    for series in time_series_list:
        scaler = MinMaxScaler()
        scaled_series = scaler.fit_transform(series)
        scalers.append(scaler)
        scaled_series_list.append(scaled_series)
    return scaled_series_list, scalers

# Transform a single time series into input-output samples
def transform_time_series_to_samples(series, num_samples=1000, input_steps=45, output_steps=14):
    total_steps = input_steps + output_steps
    num_rows = series.shape[0] - total_steps + 1
    indices = np.random.choice(num_rows, num_samples, replace=False)

    X = []
    y = []
    for idx in indices:
        X.append(series[idx : idx + input_steps])
        y.append(series[idx + input_steps : idx + total_steps])
    return np.array(X), np.array(y)

# Split data for training and testing
def split_data(X_list, y_list, engine_data, test_size=0.2, random_state=42):
    # Split each weather time series
    X_train_list, X_test_list, y_train_list, y_test_list = [], [], [], []
    for X, y in zip(X_list, y_list):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)

    # Split engine data
    engine_train, engine_test = train_test_split(engine_data, test_size=test_size, random_state=random_state)

    return X_train_list, X_test_list, y_train_list, y_test_list, engine_train, engine_test




In [5]:
# Main script
data_dir = "synthetic_data"
dummy_engine_data = "dummy_engine_data.csv"

# Load weather data as separate time series
weather_time_series = load_weather_data_as_timeseries(data_dir)
print("Loaded weather time series:", [ts.shape for ts in weather_time_series])

# Load engine data
engine_data = load_engine_data(data_dir, dummy_engine_data)
print("Engine data loaded:", engine_data.shape)

# Scale each weather time series
scaled_weather_data, weather_scalers = scale_time_series(weather_time_series)
print("Scaled weather time series.")

# Transform each weather time series into input-output samples
weather_X_list, weather_y_list = [], []
for series in scaled_weather_data:
    X, y = transform_time_series_to_samples(series, num_samples=1000, input_steps=45, output_steps=14)
    weather_X_list.append(X)
    weather_y_list.append(y)

# Combine engine variables into a single array for splitting
engine_data_combined = engine_data.to_numpy()

# Split the data
X_train_list, X_test_list, y_train_list, y_test_list, engine_train, engine_test = split_data(
    weather_X_list, weather_y_list, engine_data_combined, test_size=0.2, random_state=42
)

resistance_train, power_train, torque_train, total_time_train, distance_train, total_energy_train = (
    engine_train[:, 0], engine_train[:, 1], engine_train[:, 2], engine_train[:, 3], engine_train[:, 4], engine_train[:, 5]
)

resistance_test, power_test, torque_test, total_time_test, distance_test, total_energy_test = (
    engine_test[:, 0], engine_test[:, 1], engine_test[:, 2], engine_test[:, 3], engine_test[:, 4], engine_test[:, 5]
)

# Convert to tensors
X_train_tensors = [torch.tensor(X, dtype=torch.float32) for X in X_train_list]
X_test_tensors = [torch.tensor(X, dtype=torch.float32) for X in X_test_list]

resistance_train_tensor = torch.tensor(resistance_train, dtype=torch.float32)
power_train_tensor = torch.tensor(power_train, dtype=torch.float32)
torque_train_tensor = torch.tensor(torque_train, dtype=torch.float32)
total_time_train_tensor = torch.tensor(total_time_train, dtype=torch.float32)
distance_train_tensor = torch.tensor(distance_train, dtype=torch.float32)
total_energy_train_tensor = torch.tensor(total_energy_train, dtype=torch.float32)

resistance_train_tensor=resistance_train_tensor.unsqueeze(1)
power_train_tensor=power_train_tensor.unsqueeze(1)
torque_train_tensor=torque_train_tensor.unsqueeze(1)
total_time_train_tensor=total_time_train_tensor.unsqueeze(1)
distance_train_tensor=distance_train_tensor.unsqueeze(1)
total_energy_train_tensor = total_energy_train_tensor.unsqueeze(1)

resistance_test_tensor = torch.tensor(resistance_test, dtype=torch.float32)
power_test_tensor = torch.tensor(power_test, dtype=torch.float32)
torque_test_tensor = torch.tensor(torque_test, dtype=torch.float32)
total_time_test_tensor = torch.tensor(total_time_test, dtype=torch.float32)
distance_test_tensor = torch.tensor(distance_test, dtype=torch.float32)
total_energy_test_tensor = torch.tensor(total_energy_test, dtype=torch.float32)

X_train_tensors= torch.stack(X_train_tensors, dim=2)
X_test_tensors = torch.stack(X_test_tensors, dim=2)
print(X_train_tensors.shape)
print(f"Engine train shape: {total_energy_train_tensor.shape}, Engine test shape: {total_energy_test_tensor.shape}")


Loaded weather time series: [(1460, 10), (1460, 10), (1460, 10), (1460, 10)]
Engine data loaded: (200, 6)
Scaled weather time series.
torch.Size([800, 45, 4, 10])
Engine train shape: torch.Size([160, 1]), Engine test shape: torch.Size([40])


# MODEL DEFINITION

In [6]:
class ForecastModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ForecastModel, self).__init__()
        self.encoder_gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.decoder_gru = nn.GRU(output_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, forecast_steps):
        batch_size, time_steps, num_locations, num_features = x.size()
        
        # Initialize an empty tensor for storing forecasts across locations
        forecasts = []

        for location in range(num_locations):
            # Process each location independently
            location_input = x[:, :, location, :]  # Shape: (batch_size, time_steps, num_features)
            
            # Encoder
            _, hidden = self.encoder_gru(location_input)
            
            # Decoder
            decoder_input = torch.zeros(batch_size, 1, num_features).to(x.device)
            outputs = []
            
            for _ in range(forecast_steps):
                out, hidden = self.decoder_gru(decoder_input, hidden)
                out = self.fc(out.squeeze(1))  # Shape: (batch_size, output_size)
                outputs.append(out)
                decoder_input = out.unsqueeze(1)
            
            # Concatenate outputs for each forecast step
            location_forecast = torch.stack(outputs, dim=1)  # Shape: (batch_size, forecast_steps, num_features)
            forecasts.append(location_forecast)

        # Stack forecasts for all locations
        forecasts = torch.stack(forecasts, dim=2)  # Shape: (batch_size, forecast_steps, num_locations, num_features)
        return forecasts

# Define the combined DNN model for total energy prediction
class EnergyPredictionModel(nn.Module):
    def __init__(self, forecast_output_size, static_input_size, hidden_size):
        super(EnergyPredictionModel, self).__init__()
        self.forecast_model = ForecastModel(input_size=num_variables, hidden_size=hidden_size, output_size=forecast_output_size)
        
        # Feed-forward network for total energy prediction
        self.fc1 = nn.Linear(forecast_output_size * input_steps * num_files + static_input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Predict a single total energy value
    
    def forward(self, x_seq, resistance, power, torque, time_total, distance, forecast_steps):
        # Forecast model to get the output sequence
        forecast_out = self.forecast_model(x_seq, forecast_steps)  # Shape: (batch_size, forecast_steps, num_locations, num_features)
        forecast_out_flat = forecast_out.view(forecast_out.size(0), -1)  # Flatten the forecast output across locations and time steps
        # Repeat static inputs to match flattened forecast dimensions
        static_inputs = torch.cat((resistance, power, torque, time_total, distance), dim=1)  # Shape: (batch_size, 3)
        print(forecast_out_flat.shape)
        print(static_inputs.shape)
        static_inputs=static_inputs.squeeze(1)
        print(static_inputs.shape)
        # Concatenate static inputs with the flattened forecast output
        combined_input = torch.cat((forecast_out_flat, static_inputs), dim=1)
        
        # Feed-forward layers for total energy prediction
        x = torch.relu(self.fc1(combined_input))
        x = torch.relu(self.fc2(x))
        energy_pred = self.fc3(x)  # Output: (batch_size, 1)
        
        return energy_pred

# Initialize the model, loss function, and optimizer
model = EnergyPredictionModel(forecast_output_size=num_variables, static_input_size=5, hidden_size=64)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

TRAINER

In [7]:


# num_epochs = 10
# for epoch in range(num_epochs):
#     # Forward pass
#     # print(type(X_train_tensors))
#     # print(X_train_tensors.shape)
#     energy_pred = model(X_train_tensors, resistance_train_tensor, power_train_tensor, torque_train_tensor, total_time_train_tensor, distance_train_tensor, input_steps)
#     loss = criterion(energy_pred, total_energy_train_tensor)  # Adjusted to match prediction dimensions
    
#     # Backward and optimize
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
    
#     print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

## Trainer

In [8]:


# Parameters
num_features = 10       # Number of input features in the sequence
time_steps = 45         # Time steps in the input sequence
forecast_steps = 10     # Number of forecast steps (output sequence length)
num_locations = 4       # Number of locations
hidden_size = 64        # GRU hidden size
num_samples = 1000      # Number of data samples

# Generate random data
np.random.seed(42)
X_sequence = np.random.rand(num_samples, time_steps, num_locations, num_features).astype(np.float32)
resistance = np.random.rand(num_samples, 1).astype(np.float32)  # Single value per sample
power = np.random.rand(num_samples, 1).astype(np.float32)       # Single value per sample
torque = np.random.rand(num_samples, 1).astype(np.float32)      # Single value per sample
total_energy = np.random.rand(num_samples, 1).astype(np.float32)  # Single value per sample

# Convert data to PyTorch tensors
X_sequence = torch.tensor(X_sequence)
resistance = torch.tensor(resistance)
power = torch.tensor(power)
torque = torch.tensor(torque)
total_energy = torch.tensor(total_energy)

# Define the Seq2Seq Forecast model with GRU encoder and decoder
class ForecastModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ForecastModel, self).__init__()
        self.encoder_gru = nn.GRU(input_size, hidden_size, batch_first=True)
        self.decoder_gru = nn.GRU(output_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x, forecast_steps):
        batch_size, time_steps, num_locations, num_features = x.size()
        
        # Initialize an empty tensor for storing forecasts across locations
        forecasts = []

        for location in range(num_locations):
            # Process each location independently
            location_input = x[:, :, location, :]  # Shape: (batch_size, time_steps, num_features)
            
            # Encoder
            _, hidden = self.encoder_gru(location_input)
            
            # Decoder
            decoder_input = torch.zeros(batch_size, 1, num_features).to(x.device)
            outputs = []
            
            for _ in range(forecast_steps):
                out, hidden = self.decoder_gru(decoder_input, hidden)
                out = self.fc(out.squeeze(1))  # Shape: (batch_size, output_size)
                outputs.append(out)
                decoder_input = out.unsqueeze(1)
            
            # Concatenate outputs for each forecast step
            location_forecast = torch.stack(outputs, dim=1)  # Shape: (batch_size, forecast_steps, num_features)
            forecasts.append(location_forecast)

        # Stack forecasts for all locations
        forecasts = torch.stack(forecasts, dim=2)  # Shape: (batch_size, forecast_steps, num_locations, num_features)
        return forecasts

# Define the combined DNN model for total energy prediction
class EnergyPredictionModel(nn.Module):
    def __init__(self, forecast_output_size, static_input_size, hidden_size):
        super(EnergyPredictionModel, self).__init__()
        self.forecast_model = ForecastModel(input_size=num_features, hidden_size=hidden_size, output_size=forecast_output_size)
        
        # Feed-forward network for total energy prediction
        self.fc1 = nn.Linear(forecast_output_size * forecast_steps * num_locations + static_input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Predict a single total energy value
    
    def forward(self, x_seq, resistance, power, torque, forecast_steps):
        # Forecast model to get the output sequence
        forecast_out = self.forecast_model(x_seq, forecast_steps)  # Shape: (batch_size, forecast_steps, num_locations, num_features)
        forecast_out_flat = forecast_out.view(forecast_out.size(0), -1)  # Flatten the forecast output across locations and time steps
        print(resistance.shape)
        print(power.shape)
        print(torque.shape)
        # Repeat static inputs to match flattened forecast dimensions
        static_inputs = torch.cat((resistance, power, torque), dim=1)  # Shape: (batch_size, 3)
        
        # Concatenate static inputs with the flattened forecast output
        combined_input = torch.cat((forecast_out_flat, static_inputs), dim=1)
        
        # Feed-forward layers for total energy prediction
        x = torch.relu(self.fc1(combined_input))
        x = torch.relu(self.fc2(x))
        energy_pred = self.fc3(x)  # Output: (batch_size, 1)
        
        return energy_pred

# Initialize the model, loss function, and optimizer
model = EnergyPredictionModel(forecast_output_size=num_features, static_input_size=3, hidden_size=hidden_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    energy_pred = model(X_sequence, resistance, power, torque, forecast_steps)
    loss = criterion(energy_pred, total_energy)  # Adjusted to match prediction dimensions
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")



torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [1/10], Loss: 0.4080
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [2/10], Loss: 0.3536
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [3/10], Loss: 0.3104
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [4/10], Loss: 0.2680
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [5/10], Loss: 0.2270
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [6/10], Loss: 0.1879
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [7/10], Loss: 0.1528
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [8/10], Loss: 0.1205
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [9/10], Loss: 0.0956
torch.Size([1000, 1])
torch.Size([1000, 1])
torch.Size([1000, 1])
Epoch [10/10], Loss: 0.0823


## Output Visualization